In [3]:
pip install pysnmp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.0/80.0 kB 1.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [35]:
from pysnmp.hlapi import *

# SNMP parameters
community = 'public'
host = '192.168.1.8'  
port = 161
timeout = 1
retries = 5
#get method
def snmp_get(oid):
    error_indication, error_status, error_index, var_binds = next(
        getCmd(SnmpEngine(),
               CommunityData(community),
               UdpTransportTarget((host, port), timeout=timeout, retries=retries),
               ContextData(),
               ObjectType(ObjectIdentity(oid)))
    )
    
    if error_indication:
        print('SNMP GET error: %s' % error_indication)
        return None
    elif error_status:
        print('SNMP GET error: %s at %s' % (
            error_status.prettyPrint(),
            error_index and var_binds[int(error_index)-1][0] or '?'
            )
        )
        return None
    else:
        for varBind in var_binds:
            print(varBind[1].prettyPrint())  # Print all results
#set method
def snmp_set(oid, value, community_write='public2', data_type='str'):
    if data_type == 'str':
        value = OctetString(value)
    elif data_type == 'int':
        value = Integer(value)
    elif data_type == 'ip':
        value = IpAddress(value)
    elif data_type == 'oid':
        value = ObjectIdentifier(value)
    else:
        print('Unsupported data type')
        return
    
    error_indication, error_status, error_index, var_binds = next(
        setCmd(SnmpEngine(),
               CommunityData(community_write),  # Change community string here
               UdpTransportTarget((host, port), timeout=timeout, retries=retries),
               ContextData(),
               ObjectType(ObjectIdentity(oid), value))
    )
    
    if error_indication:
        print('SNMP SET error: %s' % error_indication)
        return False
    elif error_status:
        print('SNMP SET error: %s at %s' % (
            error_status.prettyPrint(),
            error_index and var_binds[int(error_index)-1][0] or '?'
            )
        )
        return False
    else:
        return True
#get_next method
def get_next(community, host, oid):
    iterator = nextCmd(SnmpEngine(),
                       CommunityData(community),
                       UdpTransportTarget((host, 161)),
                       ContextData(),
                       ObjectType(ObjectIdentity(oid)))

    errorIndication, errorStatus, errorIndex, varBinds = next(iterator)

    if errorIndication:
        print(errorIndication)
    elif errorStatus:
        print('%s at %s' % (
            errorStatus.prettyPrint(),
            errorIndex and varBinds[int(errorIndex) - 1][0] or '?'
        ))
    else:
        for varBind in varBinds:
            print(' = '.join([x.prettyPrint() for x in varBind]))

#main
if __name__ == "__main__":
    # Test SNMP GET operation
    print("Testing SNMP GET operation:")
    print("sysDescr:")
     # OID for sysDescr
    snmp_get('1.3.6.1.2.1.1.1.0') 
    
   
    # Set the sysName >> "Network Administrator"
    print("\nTesting SNMP SET operation:")
    print("Setting sysName to 'Network Administrator':", snmp_set('1.3.6.1.2.1.1.5.0', 'Network Administrator'))
    
    # Test SNMP GETNEXT operation
    print("\nTesting SNMP GETNEXT operation:")
     # OID for sysLocation
    get_next(community, host, '.1.3.6.1.2.1.1.6') 


Testing SNMP GET operation:
sysDescr:
Hardware: Intel64 Family 6 Model 142 Stepping 12 AT/AT COMPATIBLE - Software: Windows Version 6.3 (Build 22631 Multiprocessor Free)

Testing SNMP SET operation:
Setting sysName to 'Network Administrator': True

Testing SNMP GETNEXT operation:
SNMPv2-MIB::sysLocation.0 = cairo 
